### Data Downloading, Splitting (Asante only), and Checking of Count

In [3]:
# train_data = dataset["train"]
# dataset["train"][0]
# val_data = dataset["validation"]
# dataset["validation"][0]
# test_data = dataset["test"]
# dataset["test"][0]

# dataset["train"].to_pandas().to_csv("C:\Users\T-Plug\Desktop\University\Level 300\2nd Sem\316 Social Media Mining\Datasets\twi_train.csv", index=False)
# dataset["dev"].to_pandas().to_csv("C:\Users\T-Plug\Desktop\University\Level 300\2nd Sem\316 Social Media Mining\Datasets\twi_dev.csv", index=False)
# dataset["test"].to_pandas().to_csv("C:\Users\T-Plug\Desktop\University\Level 300\2nd Sem\316 Social Media Mining\Datasets\twi_test.csv", index=False)

# print(f"Train size: {len(train_data)}")
# print(f"Validation size: {len(val_data)}")
# print(f"Test size: {len(test_data)}")

In [12]:
import pandas as pd

In [8]:
# Load directly from GitHub
train_url = "https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/twi/train.tsv"
dev_url = "https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/twi/dev.tsv"
test_url = "https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/twi/test.tsv"

train_df = pd.read_csv(train_url, sep='\t')
print(train_df["label"].value_counts())

dev_df = pd.read_csv(dev_url, sep='\t')
print(dev_df["label"].value_counts())

test_df = pd.read_csv(test_url, sep='\t')
print(test_df["label"].value_counts())

label
positive    1644
negative    1315
neutral      522
Name: count, dtype: int64
label
positive    183
negative    147
neutral      58
Name: count, dtype: int64
label
positive    450
negative    353
neutral     146
Name: count, dtype: int64


In [11]:
# Save locally if needed
train_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_train.csv", index=False)
dev_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_dev.csv", index=False)
test_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_test.csv", index=False)

In [23]:
import chardet

def detect_encoding(file_path, num_bytes=10000):
    with open(file_path, 'rb') as f:
        raw_data = f.read(num_bytes)
    result = chardet.detect(raw_data)
    print(f"{file_path} likely encoding: {result['encoding']} (confidence: {result['confidence']})")
    return result['encoding']

# Example
encoding = detect_encoding("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/labeledl_twi_tweets.csv")


C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/labeledl_twi_tweets.csv likely encoding: Windows-1252 (confidence: 0.73)


In [24]:
df = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/labeledl_twi_tweets.csv", encoding="Windows-1252")

print(df["label"].value_counts())

label
Positive               3542
Neutral                1308
Negative               1064
No Sentiment             49
Positive & Negative      26
Multilingual              3
Twi                       3
Ghanaian Pidgin           1
Name: count, dtype: int64


In [25]:
valid_labels = ["Positive", "Neutral", "Negative"]
df_filtered = df[df["label"].isin(valid_labels)]

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
# First split: train vs temp (dev + test)
asante_train_df, temp_df = train_test_split(df_filtered, test_size=0.3, random_state=42, stratify=df_filtered["label"])

# Second split: dev vs test
asante_dev_df, asante_test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

In [28]:
# Save each split to CSV
asante_train_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_train.csv", index=False)
asante_dev_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_dev.csv", index=False)
asante_test_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_test.csv", index=False)

In [29]:
print(asante_train_df["label"].value_counts())
print(asante_dev_df["label"].value_counts())
print(asante_test_df["label"].value_counts())

label
Positive    2479
Neutral      915
Negative     745
Name: count, dtype: int64
label
Positive    531
Neutral     196
Negative    160
Name: count, dtype: int64
label
Positive    532
Neutral     197
Negative    159
Name: count, dtype: int64


### Combining of Split Asante and Already Split AfriSenti

In [32]:
# df[label_col] = df[label_col].str.strip().str.lower()  # makes 'Neutral', ' neutral ' -> 'neutral'

# loading in two-three different encodings to be able to read the emojies
import pandas as pd

def load_and_normalize(path, label_col="label"):
    encodings_to_try = ["utf-8", "windows-1252", "windows-1254"]
    
    for enc in encodings_to_try:
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except UnicodeDecodeError:
            continue
    else:
        raise ValueError(f"Failed to decode {path} with tried encodings: {encodings_to_try}")

    # Drop rows where either 'label' or 'tweet' is missing
    df = df.dropna(subset=[label_col, "tweet"])
    
    # Normalize label column
    if label_col in df.columns:
        df[label_col] = df[label_col].astype(str).str.strip().str.lower()  # makes 'Neutral', ' neutral ' -> 'neutral'
    return df

In [33]:
afri_train = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_train.csv")
afri_dev   = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_dev.csv")
afri_test  = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/afrisenti_twi_test.csv")

asante_train = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_train.csv")
asante_dev   = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_dev.csv")
asante_test  = load_and_normalize("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/asante_twi_test.csv")

# Combine into three final DataFrames
train_df = pd.concat([afri_train, asante_train], ignore_index=True)
dev_df   = pd.concat([afri_dev, asante_dev], ignore_index=True)
test_df  = pd.concat([afri_test, asante_test], ignore_index=True)

train_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_train.csv", index=False)
dev_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_dev.csv", index=False)
test_df.to_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_test.csv", index=False)


#### Emoji Handling (converting to utf-8 and and converting emojies to text labels)

In [3]:
# Printing only 'EMOJI' tweets
import re

emoji_pattern = re.compile("[\U00010000-\U0010ffff]", flags=re.UNICODE)

emoji_tweets = train_df[train_df["tweet"].apply(lambda x: bool(emoji_pattern.search(x)))]
print(emoji_tweets["tweet"].to_list())

emoji_tweets2 = dev_df[dev_df["tweet"].apply(lambda x: bool(emoji_pattern.search(x)))]
print(emoji_tweets2["tweet"].to_list())

emoji_tweets3 = test_df[test_df["tweet"].apply(lambda x: bool(emoji_pattern.search(x)))]
print(emoji_tweets3["tweet"].to_list())


['enfa bi da bra 🤣🤣', 'ah😂😂😂 na wiegyimi anaaa', '1 minute biaaa na broken heart nam mu🤣🤣', 'me bo efu paa😂😂', 'wo nyɛ papa wo🤣🤣🤣', 'lionel messi papa team ben wab da 😂😂', 'wodeɛ sɛ wano checki hwɛ 😁', 'masa wo boa koraaa 😅', 'blacko gyai gyimii no cos u belong to god not karishika 😡', '3ka aba fie 🙆\u200d♂️ sika no ahye 😅', 'trumu tanka nti you are crushing 😂😂😂 ei alhaji', 'nyame b3tua wonso kasia to 😂😁😊', 'asw3 wagyimie oooo eeiiii mey3m mpo ashi me s3naa ebi didi bi akyi😂😂😂😂😂😂😂 eeiii kweku picaso', 'akkoa yi ti ate oo 😂😂😂', 's3 eny3 nsem bi a anka ka nokware s33 chelsea nso y3 team mpo anaa 🙄 🤧🤧🤧 mu sore a dede hunu nkoaa ene mun3 🤣🤣🤣🤣🤣🤣🤣🤣', 'eeii bossu obaa nti wo ka nkwasiasem 🙄', 'y3 na y3 bl3 oo 😂😂😂', 'nyansa baako sei nni ger yi trim ad3n aa 6packs and money 😂😂', 'handouts no nkoaa🤣🤣🤣 dem go tell u say ibi 7 courses na baako bia y3 40gh y33bl3 oo', '😂😂😂😂😂😂 ah kwame sabi ɛnti nkwasia keka yi wongyae', 'ade3 yi ahye me awurade🥲', 'masa yɛ stɛdi wɔ hɔ😂😂😂', 'rasta de3 anka school f

In [6]:
# Making sure everything is converted to utf-8 and there are no errors in the datasets.

import os
import pandas as pd

input_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined"
output_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder, filename)

        try:
            df = pd.read_csv(file_path, encoding="utf-8")
        except UnicodeDecodeError:
            print(f" UTF-8 failed for {filename}, trying ISO-8859-1...")
            df = pd.read_csv(file_path, encoding="ISO-8859-1")

        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False, encoding="utf-8")
        print(f" Converted {filename} to UTF-8")


 Converted combined_dev.csv to UTF-8
 Converted combined_test.csv to UTF-8
 Converted combined_train.csv to UTF-8


In [7]:
# Checking to see if ɔ and ɛ survived

for filename in os.listdir(output_folder):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(output_folder, filename), encoding="utf-8")
        contains_e = df.astype(str).apply(lambda x: x.str.contains("ɛ", na=False)).any(axis=1).sum()
        contains_o = df.astype(str).apply(lambda x: x.str.contains("ɔ", na=False)).any(axis=1).sum()
        print(f"{filename}: {contains_e} rows have 'ɛ', {contains_o} rows have 'ɔ'")


combined_dev.csv: 70 rows have 'ɛ', 29 rows have 'ɔ'
combined_test.csv: 188 rows have 'ɛ', 90 rows have 'ɔ'
combined_train.csv: 640 rows have 'ɛ', 342 rows have 'ɔ'


In [9]:
# CONVERTING EMOJIES TO TEXT LABELS
import emoji
import pandas as pd
import os

def convert_emojis_to_text(text):
    if not isinstance(text, str):
        return text
    return emoji.demojize(text, language="en")  # ❤️ -> :red_heart:

input_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8"
output_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8_Emojized"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(input_folder, filename), encoding="utf-8")

        if "tweet" in df.columns:
            df["tweet"] = df["tweet"].apply(convert_emojis_to_text)

        df.to_csv(os.path.join(output_folder, filename), index=False, encoding="utf-8")
        print(f"Converted emojis in {filename} to text labels")


Converted emojis in combined_dev.csv to text labels
Converted emojis in combined_test.csv to text labels
Converted emojis in combined_train.csv to text labels


### Augmentation of Neutral and Negative to Prevent Overfitting

In [11]:
# Run this everytime you want to use the names of the datasets, changed according to latest csv version
import pandas as pd

train_df = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8_Emojized/combined_train.csv", encoding="utf-8")
dev_df = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8_Emojized/combined_dev.csv", encoding="utf-8")
test_df = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8_Emojized/combined_test.csv", encoding="utf-8")

In [15]:
# Checking the counts before augmentation

print("Train==========")
print(train_df["label"].value_counts())
print("Dev===========")
print(dev_df["label"].value_counts())
print("Test=========")
print(test_df["label"].value_counts())


Train==========
label
positive    4123
negative    2060
neutral     1437
Name: count, dtype: int64
Dev===========
label
positive    714
negative    307
neutral     254
Name: count, dtype: int64
Test=========
label
positive    982
negative    512
neutral     343
Name: count, dtype: int64


In [41]:
slang_lexicon = {
    "chale": {"meaning": "friend / expression of emphasis", "sentiment": "neutral"},
    "charle": {"meaning": "friend / expression of emphasis", "sentiment": "neutral"},
    "shocki": {"meaning": "surprised / shocked", "sentiment": "neutral"},
    "galamsey": {"meaning": "illegal mining", "sentiment": "negative"},
    "sia": {"meaning": "foolishness", "sentiment": "negative"},
    "bl3": {"meaning": "fool", "sentiment": "negative"},
    "b3l3": {"meaning": "fool", "sentiment": "negative"},
    "trumu": {"meaning": "prostitute", "sentiment": "negative"},
    "abeg": {"meaning": "please", "sentiment": "neutral"},
    "bossu": {"meaning": "big guy / big boss", "sentiment": "neutral"},
    "akatanii": {"meaning": "american", "sentiment": "neutral"},
    "dumsor": {"meaning": "lack of power supply / blackouts", "sentiment": "neutral"},
    "akpeteshie": {"meaning": "local gin", "sentiment": "neutral"},
    "chop": {"meaning": "eat", "sentiment": "neutral"},
    "obroni": {"meaning": "foreigner / white person", "sentiment": "neutral"},
    "sakora": {"meaning": "light-skinned / bald", "sentiment": "neutral"},
    }


In [17]:
import os
import pandas as pd
import random 

lexicon = {
    "na wo dea den": ["me ho yɛ den", "wo ho ntɔ wo", "wo ho yɛ hu"],
    "gyimie saaaa na afi aso": ["gyimie saaaa", "y3 ɔkwasea saaaa na afi aso", "yɛ ɔkwasea saaa"],
    "gyaii nsem hunuu no na bu oman no yiee": ["gyae nkwaseas3m no na mo ne ɔman no nni yie", "gyae nkwaseas3m no", "gyaii nsem hunuu no na bu oman"],
    "covid19 nkoa mponi na ebola abaframu the game enter different level": ["covid19 nkoa na seesei ebola abaframu the game enter level eii", "covid19 nko ara na seesei ebola aba afra mu", "covid19 y3 den oo"],
    "is full example of fear of man and stop saman": ["y3 onipa suro ho nhwɛso a edi mũ na 3ny3 saman", "is example of onipa suro na 3ny3 ahonhommɔne", "onipa ho suro oo"],
    "aden wo y3 akwadaa": ["nea enti a woy3 abofra", "aden enti a woy3 abofra", "opanyin te s3 ɔno"],
    "dabi me dii b3go lol": ["dabi me dii paanoo lol", "dabii me di rice", "dabi me di fufu lol"],
    "e choke u anaa": ["me ho yɛ bɔne", "me ho ntɔ me", "me ho yɛ hu"],
    "nika no da me ha nso wall no": ["nka no da me ha nso wall no", "me nkutoo na mewɔ biribi a emu yɛ duru", "nanso ɛda ha ne me nso ɔfasuo no"],
    "twam wogoro twer ne": ["twam wɔgoro twɛ ne", "pass wɔn di agorɔ twɛn na", "nanso wodi agorɔ na wɔtwɛn"],
    "kraman mullion eii": ["kraman million eii", "ɔpepem akraman eii", "ɔpepem akraman eii"],
    "dabi wasori": ["me ho yɛ bɔne", "me ho ntɔ me", "me ho yɛ hu"],
    "akakasuro ne sorowisa": ["akakasuro ne soro wisa", "ehu ne ɔsoro home", "ehu ne sorowisa"],
    "ankasa nasu y din": ["nasu ankasa no yɛ din", "nasu yɛ din", "kofi yɛ din a wɔde frɛ no daa"],
    "ny saa na merep akyer": ["ɛnyɛ saa na merep akyer", "ɛnyɛ ɛno ne nea merebɔ mmɔden sɛ mɛka", "ɛno ne nea merebɔ mmɔden sɛ mɛka"],
    "natsecurity foc nso se sn": ["me ho yɛ bɔne", "me ho ntɔ me", "me ho yɛ hu"],
    "eeeii saa mennim da ??": ["eeeii enti na mennim da ??", "enti na minnim da", "enti efi bere b3n?"],
    "oh saa??": ["oh saa na ɛte??", "saa na ɛte", "ow saa"],
    "shocki me mpo??": ["shocking me mpo??", "3y3 me ahodwiriw mpo", "mpo nie"],
    "kwasia sem saa na wabodam": ["kwasiasem saa na wa bo dam", "saa ɔkwasea na ɔyɛ kraman", "kwasiasem saa na wa gyimi"],
    "wo brofo nso ny d ??": ["wo borofo nso nyɛ d ??", "wo english nso nyɛ papa koraa", "wo brofo nso nyɛ papa"],
    "k?nk?nsani tena kurom a kuro b?": ["kɔnkɔnsani tena kurom bi?", "konkonsa bi te kuro yi mu nso?", "nsɛmmɔnedi wɔ baabiara"],
    "wonni sika nanso wop? mmaa y?de anomdwa tu tonga": ["wonni sika nanso wop? mmea de wɔn ano tow tonga", "wonni sika nanso hwɛ sɛ wɔrekasa", "wonni sika nanso hwɛ wɔn"],
    "anisohye saa na wafa mu ewu": ["aniwa saa na wafa so awu", "enti na wafa so awu", "ɔnam so awu"],
    "saa ppp nsemhunu nkoaa": ["saa paa nsemhunu nkoaa", "enti nkwaseasɛm ankasa nkutoo", "enti nkwaseasɛm nkutoo"],
    "koko nkoaagh aden wondidi biom anaa": ["koko nkoaa aden wondidi biom anaa", "koko nkoaa aden wondidi", "coconut nkoaa nea enti a wonnni bio anaa"],
    "masa wo yale anaa pt": ["masa, wo yare anaa", "woyare anaa?", "so woafi w’adwene mu?"],
    "mekyiri d?kono ne nkyenam": ["me tan dɔkono ne nkyene", "me tan evil ne mpataa a wɔayam", "me tan kyenam"],
    "wo y aboa paa ???????????? wo saa akoa wei??": ["wo y boa paa ???????????? wo saa akoa no??", "wob3tumi aboa wo saa akoa no??", "wo nkoa"],
    "sia nanka yÉ›nni pilolo anaa": ["sia nanso yenni pilolo anaa", "nanso yenni pilolo sia", "nanka yenni pilolo sia"],
    "i shock sef mo na mo bl3 tschew koraaaa": ["me shock me ho sɛ wo b3l3 tschew koraa", "me shock me ho sɛ wo tschew koraa", "wo b3l3"],
    "born one 3na wu kyere wu hu saa nu eii eugenia": ["awo baako 3na wo kyere wo hunu se eii", "awo baako ɛna wo kyere wo hunu se eii", "awo ɛna baako wo kyere wo hunu sɛ eii"],
    "john kuma paaaaaaaa funeral awwwwww ghana ne gyimie": ["john kuma paaaaaaaa ayie awwwwww ghana ne nkwasea", "ghana ne nkwaseafoɔ", "aww ayie"],
    "yakubu s3 preko y3 haram nanso É”di trumu eiii": ["yakubu s3 preko y3 haram", "preko y3 haram nanso É”di trumu", "É”di trumu eiii"],
    "firi y3n so k": ["gyae yɛn", "firi yɛn so kɔ", "twe wo ho fi yɛn ho"],
    "sika na shisika no ashi": ["sika no akɔ", "sika no ahyew", "sika no ahyew"],
    "ohia kasa fa womuÃ£ na wo didi atem": ["ohia ka womuÃ£ ho na wo bu atÃ3n", "wo bu atɛn", "ohia ka womuÃ£ ho na wo bu atɛn"],
    "ashawofo na 3di valentine": ["ashawofo na ɛdi valentine", "nguaman di valentine", "nguaman di afahyɛ"],
}

input_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined_utf8_Emojized"
output_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Augmented"
os.makedirs(output_folder, exist_ok=True)

def simple_augment(tweet):
    """Performs very basic augmentation by replacing a phrase from lexicon randomly."""
    for phrase, aug_options in lexicon.items():
        if phrase in tweet:
            return tweet.replace(phrase, random.choice(aug_options))
    return tweet

for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(file_path)

        class_counts = df['label'].value_counts()
        print(f"\n {filename} class counts before augmentation:\n{class_counts}")

        target_count = class_counts.max()

        augmented_rows = []

        for label in ["neutral", "negative"]:
            if label in class_counts:
                deficit = target_count - class_counts[label]
                if deficit > 0:
                    subset = df[df['label'] == label]
                    for _ in range(deficit):
                        # Sample a row & augment it
                        row = subset.sample(n=1, replace=True).iloc[0]
                        augmented_tweet = simple_augment(row['tweet'])
                        augmented_rows.append({"tweet": augmented_tweet, "label": label})

        if augmented_rows:
            df = pd.concat([df, pd.DataFrame(augmented_rows)], ignore_index=True)

        print(f" {filename} class counts after augmentation:\n{df['label'].value_counts()}")

        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False, encoding="utf-8")
        print(f" Augmented {filename} saved to {output_path}")



 combined_dev.csv class counts before augmentation:
label
positive    714
negative    307
neutral     254
Name: count, dtype: int64
 combined_dev.csv class counts after augmentation:
label
negative    714
neutral     714
positive    714
Name: count, dtype: int64
 Augmented combined_dev.csv saved to C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Augmented\combined_dev.csv

 combined_test.csv class counts before augmentation:
label
positive    982
negative    512
neutral     343
Name: count, dtype: int64
 combined_test.csv class counts after augmentation:
label
positive    982
negative    982
neutral     982
Name: count, dtype: int64
 Augmented combined_test.csv saved to C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Augmented\combined_test.csv

 combined_train.csv class counts before augmentation:
label
positive    4123
negative    2060
neutral     1437
Name: count, dtype

### Text Normalisation

In [18]:
import os
import pandas as pd
import unicodedata
import re

input_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Augmented"
output_folder = "C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Normalised"
os.makedirs(output_folder, exist_ok=True)

def normalize_twi_text(text):
    """Normalize Twi text while preserving ɛ and ɔ."""
    if not isinstance(text, str):
        return text

    # Step 1: Canonical Unicode normalization
    text = unicodedata.normalize("NFC", text)

    # Step 2: Remove tone marks / combining diacritics (OPTIONAL)
    # preserves ɛ, ɔ but strips extra accent marks
    text = ''.join(
        c for c in text if not unicodedata.combining(c)
    )

    # Step 3: Lowercase for consistency
    text = text.lower()

    # Step 4: Standardize whitespace (remove multiple spaces, trim)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(file_path, encoding="utf-8")

        if "tweet" in df.columns:
            df["tweet"] = df["tweet"].apply(normalize_twi_text)

        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False, encoding="utf-8")
        print(f"Normalized {filename} and saved to {output_path}")


Normalized combined_dev.csv and saved to C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Normalised\combined_dev.csv
Normalized combined_test.csv and saved to C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Normalised\combined_test.csv
Normalized combined_train.csv and saved to C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Normalised\combined_train.csv
